<a href="https://colab.research.google.com/github/sbs80/py-drums/blob/master/Snare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Snare Synthesizer
A simple snare drum synthesizer written in Python. Its parameters can either be tuned by hand or machine learnt by matching to an uploaded sample file using a genetic algorithm.

The synthesizer is loosely based on the technique described in this article: https://www.soundonsound.com/techniques/synthesizing-drums-snare-drum.

The genetic algorithm is adapted from the following guide: https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6. This has since been expanded to make a Python library PyGAD https://pypi.org/project/pygad.

# Install and import packages

In [5]:
#@title Install pyo package

!pip install pyo

In [6]:
#@title Import packages

import librosa
import librosa.feature
import numpy as np
import IPython.display as ipd
import random

#import librosa.display
#import matplotlib.pyplot as plt

from pyo import *


WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter (if available).
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



#Define snare synthesizer engine

In [0]:
#@title Snare Synth

def snare_synth(args):

  # Creates and boots the pyo server.
  # Initialize the Server in offline mode.
  s = Server(duplex=0, audio="offline")
  s.setVerbosity(0)
  s.boot()

  # Controls the overall gain of the synthesizer
  s.amp = args[14]

  # Output file duration.
  dur = 1.0

  # Set recording parameters.
  s.recordOptions(dur=dur,
                  filename='synth_snare.wav',
                  fileformat=0,
                  sampletype=0)

  # Synthesis consists of 2 sine wave generators and a filtered noise generator.
  # Frequency of sine wave generators:
  ifreq1	=		args[0]*1000
  ifreq2	=		args[1]*1000

  # Define synthesis envelopes with adjustable gains and decays:
  env1 = Adsr(attack=0.005, decay=args[2], sustain=0.0, release=0.0, dur=0.5).play()
  env2 = Adsr(attack=0.005, decay=args[3], sustain=0.0, release=0.0, dur=0.5).play()
  env3 = Adsr(attack=0.005, decay=args[4], sustain=0.0, release=0.0, dur=dur).play()
  env4 = Adsr(attack=0.005, decay=args[5], sustain=0.0, release=0.0, dur=dur).play()

  qenv1 = Pow(env1, 8, mul=args[6])
  qenv2 = Pow(env2, 8, mul=args[7])
  qenv3 = Pow(env3, 8, mul=args[8])
  qenv4 = Pow(env4, 8, mul=args[9])

  # Noise generation (white)
  n1 = Noise(args[15])

  # Noise filters:
  # lowpass:
  fn1 = Biquad(n1, args[10]*10000, q=args[12]*10, type=0, mul=qenv3).out()
  # highpass:
  fn2 = Biquad(n1, args[11]*10000, q=args[13]*10, type=1, mul=qenv4).out()

  osc1 = LFO(freq=ifreq1, type=3, mul=qenv1).out()
  osc2 = LFO(freq=ifreq2, type=3, mul=qenv2).out()

  # Start rendering.
  s.start()

  # Cleanup for the next pass.
  s.shutdown()

#test_args=[0.120,0.330,0.035,0.055,0.350,0.473,0.4,0.4,0.3,0.4,0.3,0.8040,0.2423]

#snare_synth(test_args)
#ipd.Audio('test.wav')

# Match to target sound using a genetic algorithm

In [15]:
#@title Import snare sound file

#real = '/content/drive/My Drive/snare_fix/Alesis Sr16-SR16Snare08.wav'
real = '/content/drive/My Drive/snare_fix/Roland Tr-808-TR-808Snare01.wav'
ipd.Audio(real)


In [0]:
#@title Extract features of target sound and define fitness function

n_fft = 1024 #@param {type:"number"}
hop_length = 512 #@param {type:"integer"}
n_seconds = 1 #@param {type:"integer"}

y,fs = librosa.core.load(real, sr=None)

# force length of snare sound to n_seconds seconds
y = librosa.util.fix_length(y, fs*n_seconds)

# Calculate Spectrogram of "real" snare sound
S = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length, window='hann')
S_real, phase = librosa.magphase(S)

# Spectral Centroid (not currently used):
# C_real = librosa.feature.spectral_centroid(S=S_real)

# Perform a fitness calculation for an array of synthesizer parameters
def fitness_calc(input):

  fitness = np.zeros(input.shape[0])

  for ch in range(input.shape[0]):
    # Synthesize and load snare sound
    snare_synth(input[ch].tolist())
    y,fs = librosa.core.load('synth_snare.wav', sr=None)

    # Force length to n_seconds seconds
    y = librosa.util.fix_length(y, fs*n_seconds)

    # Calculate Spectrogram of synthesized snare sound
    S = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length, window='hann')
    S_synth, phase = librosa.magphase(S)
 
    # Calulate mean squared error of the synthesized snare compared to the "real" snare
    mse = ((S_synth - S_real)**2).mean()
    
    # Fitness is defined as the negative of the mean squared error
    fitness[ch] = -mse

  return fitness


In [0]:
#@title Define genetic algorithm functions
mute_probability = 0.2 #@param {type:"slider", min:0, max:1, step:0.01}
mute_max_val = 0.8 #@param {type:"slider", min:0, max:1, step:0.01}
parent_1_probability = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}

def calc_pop_fitness(pop, parents_fitness, generation, num_parents):

     # Calculate the fitness value for each synthesizer parameter set in the current population
     # If not the first generation, don't bother recalculating fitness for the "parents"
     if generation > 0 :
      fitness = np.empty( pop.shape[0])
      fitness[0:num_parents] = parents_fitness
      fitness[num_parents:] = fitness_calc(pop[parents.shape[0]:, :])
     else :
       fitness = fitness_calc(pop)
     return fitness

def select_mating_pool(pop, fitness, num_parents):

    # Select the fittest as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    parents_fitness = np.empty(num_parents)

    for parent_num in range(num_parents):

        max_fitness_idx = np.where(fitness == np.max(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        # print(max_fitness_idx)
        # if parent_num == 0:
          # print(pop[max_fitness_idx, :])


        parents[parent_num, :] = pop[max_fitness_idx, :]
        parents_fitness[parent_num] = fitness[max_fitness_idx]

        fitness[max_fitness_idx] = -99999999999

    return parents, parents_fitness

def crossover(parents, offspring_size):

     offspring = np.empty(offspring_size)

     for k in range(offspring_size[0]):
         # Index of the first parent to mate.
         parent1_idx = k%parents.shape[0]
         # Index of the second parent to mate.
         parent2_idx = (k+1)%parents.shape[0]

         for l in range(offspring_size[1]):
             if np.random.uniform() < parent_1_probability:
               offspring[k, l] = parents[parent1_idx, l]
             else:
               offspring[k, l] = parents[parent2_idx, l]
     return offspring

def mutation(offspring_crossover,num_weights):

    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):

        # Randomly mutate some genes
        for weight in range(num_weights):
          if np.random.uniform() < mute_probability:
            random_value = np.random.uniform(-mute_max_val, mute_max_val, 1)

            # Apply mutation
            offspring_crossover[idx, weight] = offspring_crossover[idx, weight] + random_value

    return offspring_crossover

In [39]:
#@title Run genetic algorithm
num_population = 30 #@param {type:"integer"}
num_generations = 100 #@param {type:"integer"}
num_parents_mating = 5 #@param {type:"integer"}

num_params = 16

# Calculate size of array need to hold entire population of parameters
pop_size = (num_population,num_params) 

#Create an initial random population
new_population = np.random.uniform(low=0.0, high=1.0, size=pop_size)

parents_fitness = np.empty(num_parents_mating)




for generation in range(num_generations):
     
     # Measure the fitness of each member in the population.
     fitness = calc_pop_fitness(new_population, parents_fitness, generation, num_parents_mating)
     
     print("Generation " + str(generation) + " best score: " + str(fitness.max()))

     # Select the best parents in the population for mating.
     parents, parents_fitness = select_mating_pool(new_population, fitness, num_parents_mating)
 
     # Generate the                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                next generation using crossover.
     offspring_crossover = crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], num_params))
 
     # Adding some variations to the offsrping using mutation.
     offspring_mutation = mutation(offspring_crossover,num_params)
      # Creating the new population based on the parents and offspring.
     new_population[0:parents.shape[0], :] = parents
     new_population[parents.shape[0]:, :] = offspring_mutation


Generation 0 best score: -1.039133906364441
Generation 1 best score: -1.010338544845581
Generation 2 best score: -0.9963979125022888
Generation 3 best score: -0.9619691967964172
Generation 4 best score: -0.9197074770927429
Generation 5 best score: -0.9197074770927429
Generation 6 best score: -0.9157849550247192
Generation 7 best score: -0.9087010025978088
Generation 8 best score: -0.9063580632209778
Generation 9 best score: -0.8794566988945007
Generation 10 best score: -0.8710801005363464
Generation 11 best score: -0.8599511384963989
Generation 12 best score: -0.8227654099464417
Generation 13 best score: -0.8227654099464417
Generation 14 best score: -0.8227654099464417
Generation 15 best score: -0.8227654099464417
Generation 16 best score: -0.8227654099464417
Generation 17 best score: -0.8129777908325195
Generation 18 best score: -0.7992334961891174
Generation 19 best score: -0.7992334961891174
Generation 20 best score: -0.7992334961891174
Generation 21 best score: -0.7992334961891174


In [41]:
#@title Listen to results!

print("Generation number reached: " + str(generation))

print("Target sample:")
ipd.display(ipd.Audio(real))

print("Closest match according to fitness function:")
max_fitness_idx = np.where(parents_fitness == np.max(parents_fitness))
max_fitness_idx = max_fitness_idx[0][0]
snare_synth(new_population[max_fitness_idx].tolist())
ipd.display(ipd.Audio('synth_snare.wav'))
print("parameters used to achieve closest match:")
print(new_population[max_fitness_idx])

print("Randomly generated sample for comparison:")
random_params = np.random.uniform(low=0.0, high=1.0, size=num_params)
snare_synth(random_params.tolist())
ipd.display(ipd.Audio('synth_snare.wav'))


Generation number reached: 99
Target sample:


Closest match according to fitness function:


parameters used to achieve closest match:
[ 0.04673079  0.30825311 -1.23765866 -0.40247754  0.22541778 -0.10647904
  1.86389112  0.69924886  2.81116873  1.22009877  0.42729575  0.3456993
  0.15128154  0.66376392  0.83246412  0.57629813]
Randomly generated sample for comparison:


# Or manually define drum synthesizer parameters...

In [40]:
#@title Snare Synth Manual Edit
sine_1_freq = 0.53 #@param {type:"slider", min:0, max:1, step:0.01}
sine_2_freq = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
sine_1_decay = 0.56 #@param {type:"slider", min:0, max:1, step:0.01}
sine_2_decay = 0.21 #@param {type:"slider", min:0, max:1, step:0.01}
noise_lowpass_decay = 0.17 #@param {type:"slider", min:0, max:1, step:0.01}
noise_highpass_decay = 0.4 #@param {type:"slider", min:0, max:1, step:0.01}
sine_1_gain = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
sine_2_gain = 0.27 #@param {type:"slider", min:0, max:1, step:0.01}
noise_lowpass_gain = 0.61 #@param {type:"slider", min:0, max:1, step:0.01}
noise_highpass_gain = 0.43 #@param {type:"slider", min:0, max:1, step:0.01}
noise_lowpass_freq = 0.21 #@param {type:"slider", min:0, max:1, step:0.01}
noise_highpass_freq = 0.43 #@param {type:"slider", min:0, max:1, step:0.01}
noise_lowpass_q = 0.47 #@param {type:"slider", min:0, max:1, step:0.01}
noise_highpass_q = 0.43 #@param {type:"slider", min:0, max:1, step:0.01}
output_gain = 0.83 #@param {type:"slider", min:0, max:1, step:0.01}

args=[
      sine_1_freq,
      sine_2_freq,
      sine_1_decay,
      sine_2_decay,
      noise_lowpass_decay,
      noise_highpass_decay,
      sine_1_gain,
      sine_2_gain,
      noise_lowpass_gain,
      noise_highpass_gain,
      noise_lowpass_freq,
      noise_highpass_freq,
      noise_lowpass_q,
      noise_highpass_q,
      output_gain
      ]

snare_synth(args)
ipd.Audio('synth_snare.wav')

IndexError: ignored

In [0]:

#class Model(object):
#  def __init__(self):
    # Initialize the weights to `5.0` and the bias to `0.0`
    # In practice, these should be initialized to random values (for example, with `tf.random.normal`)
#    self.p = tf.Variable(tf.random.uniform([13]))

#  def __call__(self):
#    print(self.p.numpy())
#    snare_synth(self.p.numpy().tolist())

#inputp=np.random.rand(13)
#params=np.random.rand(13)

#print(params)

#ipd.display(ipd.Audio('test.wav'))

#model=Model()

#opt=tf.keras.optimizers.SGD()

#p = tf.Variable(tf.random.uniform([13]),trainable=True)
#print(p)

#q = tf.Variable(tf.random.uniform([13]),trainable=True)
#print(q)

#loss_fn = lambda: loss_calc(p)
#var_list_fn = lambda: p

#loss_test=loss_calc()
#print(loss_test)



#with tf.GradientTape() as tape:
    #losso = loss_calc(p)
    #tape.watch(p)
    #losso = tf.cast(loss_calc(p),dtype=tf.float32)
#    snare_synth(params.tolist())
#    losso = tf.math.reduce_sum(p)

#print('Var: ', p)
#print('poo: ', losso)
#gradients = tape.gradient(losso, p)
#print('Gradients: ', gradients)

#print(tf.trainable_variables())

#opt_op = opt.minimize(loss=loss_fn, var_list=p)
#opt_op.run()

#for i in range(5):
#  snare_synth(p.numpy().tolist())
#  opt.minimize(loss_fn, var_list_fn)
#loss_out=loss_calc(p)
#opt.get_gradients(loss_out,p)
#losses = tfp.math.minimize(loss_fn, num_steps=100, optimizer=tf.optimizers.Adam(learning_rate=0.1))


#model.fit(inputp,params,epochs=5)

#print(params)